In [13]:
import os

import sklearn
import pandas as pd

In [15]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = '1_autopilot'

print(role)
print(sess)
print(region)
print(bucket)
print(prefix)

arn:aws:iam::467432373215:role/service-role/AmazonSageMaker-ExecutionRole-20221206T164397
eu-west-1
sagemaker-eu-west-1-467432373215
1_autopilot


___

In [16]:
from sklearn.datasets import load_boston
boston = load_boston()

In [48]:
x = pd.DataFrame(data=boston['data'], columns=boston['feature_names'])
y = pd.Series(boston['target'], name = 'MEDV')


- The data need to have the target variable in the first column.
- Amazon SageMaker requires that a CSV file does not have a header record and that the target variable is in the first column.
- More in: https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html

In [49]:
data = pd.concat([y, x], axis=1)

In [54]:
data.shape

(506, 14)

## Upload files to s3
___

- For Autopilot we need the headers and no index in the csvs

In [55]:
data.to_csv('data_boston_autopilot.csv', index=False, header=True)

In [56]:
sess.upload_data(path='data_boston_autopilot.csv', bucket=bucket, key_prefix=f'{prefix}/data')

's3://sagemaker-eu-west-1-467432373215/1_autopilot/data/data_boston_autopilot.csv'

In [53]:
! aws s3 ls {bucket}/{prefix}/data --recursive

2022-12-11 10:12:29      23427 1_autopilot/data//train.csv
2022-12-11 10:17:09          0 1_autopilot/data/boston-test/validations/output_validation
2022-12-11 10:20:58      23490 1_autopilot/data/data_boston_autopilot.csv
2022-12-11 10:12:38      23427 1_autopilot/data/train.csv
2022-12-11 10:14:29      23490 1_autopilot/data/train_header.csv
2022-12-11 10:12:40       7818 1_autopilot/data/validation.csv
2022-12-11 10:14:31       7881 1_autopilot/data/validation_header.csv
